In [2]:
#Importing the libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import pyodbc
from IPython.display import display, HTML
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, explained_variance_score
from sklearn.ensemble import RandomForestRegressor
import os
from dotenv import load_dotenv

In [3]:
#To display a large dataset
pd.options.display.max_columns = 50
pd.options.display.max_rows = 500

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [4]:
# Load environment variables from .env file
load_dotenv()

#Fetching data from environment variable
server=os.getenv("server")
database=os.getenv("database")
username=os.getenv("user_name")
password=os.getenv("pass_word")

True

In [5]:
print(username)

amplifaiadmin


In [30]:
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)

In [31]:
emp_df = pd.read_sql_query('''
select distinct employeeid, hiredate, termdate from dim_employee_v3
where companyguid = '1740DFCD-EFB4-4F97-BF9A-229E737C53C3' and organization = 'TMO Business' 
''',cnxn)

perf_df=pd.read_sql_query('''
select distinct companyguid, organization, primaryprogram, employeeid, weekstartdate, metricname, metrictype, goal, actualvalue from perfmgmntweekly_metric_v3_agent
where companyguid = '1740DFCD-EFB4-4F97-BF9A-229E737C53C3' and organization = 'TMO Business' and metricname = '7D FCR%' and weekstartdate between '2022-10-01' and '2023-08-01'
and actualvalue is not null
order by employeeid, weekstartdate
''',cnxn)
cnxn.close()

C:\Users\aswin\AppData\Local\Temp\ipykernel_81720\989088481.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  emp_df = pd.read_sql_query('''
C:\Users\aswin\AppData\Local\Temp\ipykernel_81720\989088481.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  perf_df=pd.read_sql_query('''


In [9]:
#Pulling Staging data
#Fetching data from environment variable
stg_server=os.getenv("stg_server")
stg_database=os.getenv("stg_database")
stg_user_name=os.getenv("stg_user_name")
stg_pass_word=os.getenv("stg_pass_word")
conn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+stg_server+';DATABASE='+stg_database+';UID='+stg_user_name+';PWD='+ stg_pass_word)

In [10]:
call_df=pd.read_sql_query('''
SELECT *
  FROM [amplifai_prodstaging].[dbo].[IQorTMobile_PerformanceData]
  where date between '2022-10-01' and '2023-06-01'
''',conn)
conn.close()

C:\Users\aswin\AppData\Local\Temp\ipykernel_81720\2227506026.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  call_df=pd.read_sql_query('''


KeyboardInterrupt: 

In [32]:
#Merging the datasets on the employeeid column
df=pd.merge(emp_df,perf_df,on="employeeid")

In [33]:
#Converting hiredate and weekstart date into date-time type, actual value to float type 
df["weekstartdate"]=pd.to_datetime(df["weekstartdate"])
df["hiredate"]=pd.to_datetime(df["hiredate"])
df["actualvalue"]=df["actualvalue"].astype(float)

In [34]:
#Calculating tenure for each week period
df["tenure"]=(df["weekstartdate"]-df["hiredate"]).dt.days

In [35]:
#Creating tenure bucket
# 1-> 0-30, 2-> 31-60, 3-> 61-90, 4-> 91-180, 5->180+

# Define the ranges and labels for the buckets
ranges = [0, 30, 60, 90, 180, float('inf')]
labels = [1, 2, 3, 4, 5]

# Create the new column "bucket" based on the "tenure" values
df["tenure_bucket"] = pd.cut(df["tenure"], bins=ranges, labels=labels, right=False)

In [36]:
#Calculating days left in Tenure
# Define a function to calculate days_left_tenure
def calculate_days_left_tenure(row):
    if row['tenure_bucket'] == 1:
        return max(0, 30 - row['tenure'])
    elif row['tenure_bucket'] == 2:
        return max(0, 60 - row['tenure'])
    elif row['tenure_bucket'] == 3:
        return max(0, 90 - row['tenure'])
    elif row['tenure_bucket'] == 4:
        return max(0, 180 - row['tenure'])
    elif row['tenure_bucket'] == 5:
        return max(0, row['tenure'] - 180)
    else:
        return None

In [37]:
df['days_left_current_tenure'] = df.apply(calculate_days_left_tenure, axis=1)

In [38]:
#Looking at the data for the essential columns
df1=df[["employeeid","weekstartdate","tenure","tenure_bucket","days_left_current_tenure","actualvalue"]]

In [39]:
## Dropping outliers to calculate bucket persona
df1=df1[df1["actualvalue"]!=100]

In [40]:
#Calculating median persona goal for the each bucket (Top 20% median goal)
d={}
for i in df1["tenure_bucket"].unique():
    temp=df1[df1["tenure_bucket"]==i]
    threshold=temp["actualvalue"].quantile(0.8)
    median_values=temp[temp["actualvalue"]>=threshold]["actualvalue"].median()
    print("Bucket: ",i,"\t Median Goal: ",median_values)
    d[i]=median_values

Bucket:  2 	 Median Goal:  80.0
Bucket:  3 	 Median Goal:  78.95
Bucket:  4 	 Median Goal:  77.78
Bucket:  5 	 Median Goal:  76.42
Bucket:  nan 	 Median Goal:  nan


In [49]:
r={}
r[2]=76.42
r[3]=77.78
r[4]=78.95
r[5]=80.0

In [51]:
# Creating a new column for bucket persona
df1["bucket_persona"]=df1.apply(lambda row: r.get(row["tenure_bucket"]),axis=1)

In [52]:
#Calculating bucket average:
d1={}
for i in df1.tenure_bucket.unique():
    temp=df1[df1["tenure_bucket"]==i]
    d1[i]=temp["actualvalue"].mean()

In [53]:
d1

{2: 58.787085137085136,
 3: 64.9259524802001,
 4: 65.9565954868999,
 5: 66.18171918204618,
 nan: nan}

In [54]:
# Creating new column for bucket average 
df1["bucket_average"]=df1.apply(lambda row: d1.get(row["tenure_bucket"]),axis=1)

In [56]:
df1.to_csv("TargetSetting.csv",index=False)

In [59]:
df1['tenure_bucket'].unique().dropna()

[2, 3, 4, 5]
Categories (5, int64): [1 < 2 < 3 < 4 < 5]

In [62]:
# Create an empty DataFrame to store the sampled records
sampled_df = pd.DataFrame()

#Random Sampling of 125 records for each bucket
# Iterate over each unique tenure_bucket
for bucket in df1['tenure_bucket'].unique().dropna():
    # Get all records with the current tenure_bucket
    records = df1[df1['tenure_bucket'] == bucket]
    
    # Sample 125 random records
    sampled_records = records.sample(n=250, random_state=42)  # Set random_state for reproducibility
    
    # Append the sampled records to the sampled_df DataFrame
    sampled_df = sampled_df.append(sampled_records)

C:\Users\aswin\AppData\Local\Temp\ipykernel_81720\822316288.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sampled_df = sampled_df.append(sampled_records)
C:\Users\aswin\AppData\Local\Temp\ipykernel_81720\822316288.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sampled_df = sampled_df.append(sampled_records)
C:\Users\aswin\AppData\Local\Temp\ipykernel_81720\822316288.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sampled_df = sampled_df.append(sampled_records)
C:\Users\aswin\AppData\Local\Temp\ipykernel_81720\822316288.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sampled_df = sampled_df.append(sampled_records)


In [214]:
sampled_df.tenure_bucket.value_counts()

2    250
3    250
4    250
5    250
1      0
Name: tenure_bucket, dtype: int64

In [63]:
sampled_df.to_csv("Sampled_Data.csv",index=False)

In [188]:
df2=pd.read_csv("Sampled_Data_Target.csv")

In [215]:
df2.tenure_bucket.value_counts()

5    540
4    264
3    192
2      3
Name: tenure_bucket, dtype: int64

In [189]:
#Shuffling all the records in the dataframe
df2 = df2.sample(frac=1, random_state=42)  


In [190]:
df2.tenure_bucket.unique()

array([3, 5, 4, 2], dtype=int64)

In [191]:
df2.head()

,employeeid,weekstartdate,tenure,tenure_bucket,days_left_current_tenure,actualvalue,bucket_persona,bucket_average,Target
453,markryan.almosera,5/28/2023,62,3,28,75.00,77.78,64.925952,78.65000
793,maryanne.along,2/19/2023,1826,5,1646,70.53,80.00,66.181719,77.85830
209,jaybee.andaya,2/12/2023,1650,5,1470,62.57,80.00,66.181719,65.84000
309,jasper.literado,2/19/2023,1098,5,918,72.55,80.00,66.181719,75.64000
740,allyson.gatchalian,3/26/2023,139,4,41,82.61,78.95,65.956595,80.83523


In [192]:
df2=df2.dropna(axis=0)

In [193]:
df2.columns

Index(['employeeid', 'weekstartdate', 'tenure', 'tenure_bucket',
       'days_left_current_tenure', 'actualvalue', 'bucket_persona',
       'bucket_average', 'Target'],
      dtype='object')

In [213]:
df2.tenure_bucket.value_counts()

5    540
4    264
3    192
2      3
Name: tenure_bucket, dtype: int64

In [194]:
X=df2[['tenure', 'tenure_bucket',
       'days_left_current_tenure', 'actualvalue', 'bucket_persona',
       'bucket_average']]
y=df2[["Target"]]

In [195]:
#Creating train test split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.1,random_state=42)

In [196]:
from sklearn.ensemble import RandomForestRegressor
rf=RandomForestRegressor()
rf.fit(X_train,y_train)

C:\Users\aswin\AppData\Local\Temp\ipykernel_81720\579562820.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(X_train,y_train)


RandomForestRegressor()

In [197]:
y_pred=rf.predict(X_test)

In [198]:
#Evaluation of the model
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, explained_variance_score
# Calculate the evaluation metrics
mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
explained_variance = explained_variance_score(y_test, y_pred)

# Print the evaluation metrics
print("Mean Squared Error (MSE):", mse)
print("Root Mean Squared Error (RMSE):", rmse)
print("Mean Absolute Error (MAE):", mae)
print("R-squared (R²):", r2)
print("Explained Variance Score:", explained_variance)


Mean Squared Error (MSE): 17.978128399281438
Root Mean Squared Error (RMSE): 4.2400623107781605
Mean Absolute Error (MAE): 2.4682732897830966
R-squared (R²): 0.7731868498085294
Explained Variance Score: 0.7748532223738529


In [199]:
#Calulating the train and test score:
print("Train score: ",rf.score(X_train,y_train))
print("Test score: ",rf.score(X_test,y_test))

Train score:  0.9707080641891888
Test score:  0.7731868498085294


In [200]:
#Hyperparameter tuning
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

In [201]:
#Randomized Search CV

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1200, num = 12)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(5, 30, num = 6)]
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10, 15, 100]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 5, 10]

In [202]:
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}

In [203]:
rf=RandomForestRegressor()

In [204]:
rf=RandomizedSearchCV(estimator=rf,param_distributions=random_grid,scoring="neg_root_mean_squared_error",n_iter=10,cv=5,verbose=2,random_state=42)

In [205]:
rf.fit(X_train,y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


c:\Users\aswin\anaconda3\envs\chatbot\lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)


[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=5, min_samples_split=5, n_estimators=900; total time=   2.4s


c:\Users\aswin\anaconda3\envs\chatbot\lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)


[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=5, min_samples_split=5, n_estimators=900; total time=   2.0s


...

[CV] END max_depth=20, max_features=auto, min_samples_leaf=1, min_samples_split=15, n_estimators=700; total time=   2.0s


c:\Users\aswin\anaconda3\envs\chatbot\lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
c:\Users\aswin\anaconda3\envs\chatbot\lib\site-packages\sklearn\ensemble\_forest.py:414: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(


[CV] END max_depth=20, max_features=auto, min_samples_leaf=1, min_samples_split=15, n_estimators=700; total time=   2.2s


c:\Users\aswin\anaconda3\envs\chatbot\lib\site-packages\sklearn\model_selection\_search.py:910: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)
c:\Users\aswin\anaconda3\envs\chatbot\lib\site-packages\sklearn\ensemble\_forest.py:414: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(


RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(),
                   param_distributions={'max_depth': [5, 10, 15, 20, 25, 30],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 5, 10],
                                        'min_samples_split': [2, 5, 10, 15,
                                                              100],
                                        'n_estimators': [100, 200, 300, 400,
                                                         500, 600, 700, 800,
                                                         900, 1000, 1100,
                                                         1200]},
                   random_state=42, scoring='neg_root_mean_squared_error',
                   verbose=2)

In [206]:
predictions_RF=rf.predict(X_test)

In [207]:
# Calculate the evaluation metrics
mse = mean_squared_error(y_test, predictions_RF)
rmse = mean_squared_error(y_test, predictions_RF, squared=False)
mae = mean_absolute_error(y_test, predictions_RF)
r2 = r2_score(y_test, predictions_RF)
explained_variance = explained_variance_score(y_test, predictions_RF)

# Print the evaluation metrics
print("Mean Squared Error (MSE):", mse)
print("Root Mean Squared Error (RMSE):", rmse)
print("Mean Absolute Error (MAE):", mae)
print("R-squared (R²):", r2)
print("Explained Variance Score:", explained_variance)

Mean Squared Error (MSE): 9.226508658395547
Root Mean Squared Error (RMSE): 3.0375168573022844
Mean Absolute Error (MAE): 2.2239101677240893
R-squared (R²): 0.8835978113181562
Explained Variance Score: 0.8836031646661721


In [208]:
# Final DataFrame with target and pred
final=pd.DataFrame(X_test)
final["target"]=y_test
final["pred"]=y_pred
final["fine_tuned_pred"]=predictions_RF

In [161]:
final.to_csv("PredictionAfterInterchangingBucketAvg.csv")

In [209]:
final.head()

,tenure,tenure_bucket,days_left_current_tenure,actualvalue,bucket_persona,bucket_average,target,pred,fine_tuned_pred
633,153,4,27,66.67,78.95,65.956595,78.74765,69.341703,71.253199
904,1913,5,1733,50.00,80.00,66.181719,61.29480,77.627051,63.842684
826,2289,5,2109,72.41,80.00,66.181719,72.99000,74.152932,76.381493
697,139,4,41,41.67,78.95,65.956595,59.30020,53.093343,54.967871
976,1398,5,1218,72.06,80.00,66.181719,76.59958,77.927914,76.579143


In [210]:
final["tenure_bucket"].unique()

array([4, 5, 3, 2], dtype=int64)

In [211]:
final["tenure_bucket"].value_counts()

5    56
4    25
3    18
2     1
Name: tenure_bucket, dtype: int64

In [216]:
test=pd.read_csv("Sampled_Data.csv")

In [218]:
test=test[test["tenure_bucket"]==2]

In [219]:
test.columns

Index(['employeeid', 'weekstartdate', 'tenure', 'tenure_bucket',
       'days_left_current_tenure', 'actualvalue', 'bucket_persona',
       'bucket_average'],
      dtype='object')

In [220]:
test=test[['tenure', 'tenure_bucket',
       'days_left_current_tenure', 'actualvalue', 'bucket_persona',
       'bucket_average']]

In [221]:
bucket2_pred=rf.predict(test)

In [222]:
test["pred"]=bucket2_pred

In [223]:
test.head()

,tenure,tenure_bucket,days_left_current_tenure,actualvalue,bucket_persona,bucket_average,pred
0,55.0,2,5.0,88.89,76.42,58.787085,89.487812
1,55.0,2,5.0,85.71,76.42,58.787085,88.808549
2,55.0,2,5.0,73.33,76.42,58.787085,76.599012
3,55.0,2,5.0,72.73,76.42,58.787085,75.476134
4,55.0,2,5.0,66.67,76.42,58.787085,70.602741


In [224]:
test.to_csv("bucket2_pred.csv")

In [225]:
new_data=pd.read_csv("TargetSetting.csv")

In [228]:
new_data.columns

Index(['employeeid', 'weekstartdate', 'tenure', 'tenure_bucket',
       'days_left_current_tenure', 'actualvalue', 'bucket_persona',
       'bucket_average', 'Target'],
      dtype='object')

In [233]:
new_data=new_data[['tenure', 'tenure_bucket',
       'days_left_current_tenure', 'actualvalue', 'bucket_persona',
       'bucket_average']]

In [236]:
len(new_data)

40797

In [235]:
new_data=new_data.dropna(axis=0)

In [237]:
new_data_pred=rf.predict(new_data)

In [238]:
new_data["pred"]=new_data_pred

C:\Users\aswin\AppData\Local\Temp\ipykernel_81720\399636365.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data["pred"]=new_data_pred


In [239]:
new_data.to_csv("PredictForAllAgents.csv")

In [240]:
import pickle

In [241]:
with open('trained_model.pkl', 'wb') as file:
    pickle.dump(rf, file)

In [242]:
new_data.head(2)

,tenure,tenure_bucket,days_left_current_tenure,actualvalue,bucket_persona,bucket_average,pred
0,55.0,2.0,5.0,55.56,76.42,58.787085,59.009151
1,62.0,3.0,28.0,88.89,77.78,64.925952,89.487812
